In [4]:
import database as db
import ipywidgets as widgets
from ipywidgets import Button, Layout
from IPython.display import display,clear_output
from ipywidgets import Button, HBox, VBox
import traceback

dm = db.DataManager('default','./to_annotate/dev2.txt')

args_type = ['arg0','arg1','arg2','arg4','loc','tmp','prp']
arg_list = []
pred_input = widgets.Text(description='predicate',layout=Layout(width='80%'))
for arg_str in args_type:
    arg_list.append(widgets.Text(description=arg_str,layout=Layout(width='80%')))
left_box = VBox([pred_input]+arg_list[0:3],layout=Layout(width='30%'))
right_box = VBox(arg_list[3:],layout=Layout(width='30%'))
box = HBox([left_box,right_box])

def load_data(dm,sentid):
    
    data = dm.data
    punctuation = ['。', '，', '？', '&', '！', '@', '；', '（', '）', '、']
    record = data['id2sent'][sentid]
    record = record.split(' ')
    i = 0
    conversation = ''
    for word in record:
        if word in punctuation:
            conversation += word
            i += 1
            continue
        if word == '<SEP>':
            conversation += '\n'
            continue

        conversation += word+'({}) '.format(i)
        i += 1
    print(conversation)
    print(len(dm.data['done']))

def display_all():
    display(box)
    display(button_box)
    print('progress:{}/{}'.format(dm.cur_idx,len(dm.todo)))

def submit_result(b):
    id_ = dm.fetch_next()
    try:
        arg_dict = {}
        pred_idx = int(pred_input.value)
        pred_input.value = ''
        for idx,t in enumerate(arg_list):
            if t.value != '':
                val = t.value.split(',')
                val = [int(i) for i in val]
                if len(val) == 1:
                    val.append(val[0])
                assert val[1] >= val[0]
                arg_dict[args_type[idx]] = val
                t.value =  ''
        id_ = dm.fetch_next()
        dm.save(id_,pred_idx,arg_dict)
        print('just commit: pred:',pred_idx,'arg:',arg_dict)
    except:
        print('error! please examine the input format')
        traceback.print_exc()

def next_sentence(b):
    dm.commit()
    id_ = dm.fetch_next()
    clear_output()
    load_data(dm,id_)
    display_all()
    
def save(b):
    dm.data.sync()
    dm.data.close()
    clear_output()
    print('save done!')
    
submit_button = widgets.Button(description="Submit",)
next_button = widgets.Button(description='Next Sentence')
save_button = widgets.Button(description='Save and Quit')
button_box = HBox([submit_button,next_button,save_button])
next_button.on_click(next_sentence)
submit_button.on_click(submit_result)
save_button.on_click(save)

id_ = dm.fetch_next()
load_data(dm,id_)
display_all()

save done!
